In [14]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import type_score,get_year_ipcc,get_year_ipbes,get_countries,check_doi_glutton,normalize
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache

In [22]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')

In [23]:
df_ipcc=df_ipcc.dropna(subset=['doi'])

In [24]:
pd.Series([list(pd.Series([x[i].get('wg') for i in range(len(x))]).drop_duplicates()) for x in list(df_ipcc.drop_duplicates(subset=['doi']).dropna(subset=['doi']).wg_chap)]).value_counts()

[2]                   21803
[3]                   12324
[1]                   10478
[2_cross]              3491
[2, 2_cross]           1531
[1, 2]                 1459
[2, 3]                 1226
[1, 2, 2_cross]         251
[1, 3]                  189
[1, 2_cross]            176
[1, 2, 3]               172
[2, 2_cross, 3]          94
[2_cross, 3]             34
[1, 2, 2_cross, 3]       24
[1, 2_cross, 3]           6
Name: count, dtype: int64

In [6]:
min([int(x) for x in list(df_ipcc.year_OA.drop_duplicates().dropna())])

1686

In [25]:
df_ipcc.test_glutton.value_counts()

test_glutton
True    74098
Name: count, dtype: int64

In [26]:
df_ipcc=df_ipcc.rename(columns={'wg_chap':'ipcc'})

In [27]:
df_ipcc_group=df_ipcc.groupby(by=['doi','year'], dropna=True).agg({'ipcc': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'freq':'sum'})
df_ipcc_group=df_ipcc_group.reset_index()

In [28]:
df_ipcc_group

,doi,year,ipcc,countries,freq
0,10.1001/jama.2012.108940,2012.0,"[{'name': 'wg2_chap_06', 'wg': '2', 'chap': 6}]",[US],1
1,10.1001/jama.2014.13186,2014.0,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7}]","[US, GB]",1
2,10.1001/jama.2014.15715,2014.0,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7}]",[US],1
3,10.1001/jama.2020.1097,2020.0,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7}]",[US],1
4,10.1001/jama.2020.15398,2020.0,"[{'name': 'wg2_chap_04', 'wg': '2', 'chap': 4}...",[US],3
...,...,...,...,...,...
53411,10.9774/gleaf.9781315651354,2017.0,"[{'name': 'wg3_chap_01', 'wg': '3', 'chap': 1}]",[None],1
53412,10.9778/cmajo.20170047,2017.0,"[{'name': 'wg2_chap_14', 'wg': '2', 'chap': 14}]",[CA],1
53413,10.9783/9780812205381.291,2006.0,"[{'name': 'wg3_chap_14', 'wg': '3', 'chap': 14}]",[None],1
53414,10.9783/9780812205381.336,2006.0,"[{'name': 'wg3_chap_14', 'wg': '3', 'chap': 14}]",[None],1


In [30]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [36]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization2.json')

In [41]:
df_ipbes.columns

Index(['doi', 'date', 'countries', 'concepts', 'sdg', 'topics',
       'is_OA_available', 'year_OA', 'authors_name', 'rors',
       'institutions_names', 'locations_names', 'year'],
      dtype='object')

In [32]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: [{'chapter': str(x['tag']).replace('ipbes-global_chapter','')} for x in row['tags']], axis=1)

In [33]:
df_ipbes_group=df_ipbes.groupby(by=['doi','year'], dropna=True).agg({'ipbes': lambda x: list(x)[0], 'countries': lambda x: list(x), 'freq': 'sum'})
df_ipbes_group=df_ipbes_group.reset_index()

KeyError: 'year'

In [ ]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: x[0])
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [ ]:
df_ipbes['year']=df_ipbes.loc[:,'date'].apply(lambda x: get_year_ipbes(str(x)) if pd.isna(x)==False else None)

In [ ]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')
df_ipcc_ipbes

In [ ]:
update_bso_publications()

In [ ]:
bso=pd.DataFrame(get_bso_publications())

In [ ]:
df_bso=bso.transpose().reset_index()

In [ ]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

In [ ]:
df_bso

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left')

In [ ]:
df_ipcc_ipbes_bso['countries']=df_ipcc_ipbes_bso.apply(lambda row: get_countries(row),axis=1)

In [ ]:
df_ipcc_ipbes_bso=df_ipcc_ipbes_bso[['doi', 'ipcc', 'ipbes', 'countries', 'rors', 'bso_local_affiliations']]

In [ ]:
df_iso=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\iso_countries.json')

In [ ]:
df_iso=df_iso.rename(columns={'code':'countries'})

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_iso, on='countries', how='left')
del df_ipcc_ipbes_bso['countries']
df_ipcc_ipbes_bso=df_ipcc_ipbes_bso.rename(columns={'name':'countries'})

In [ ]:
df_brevet_doi=pd.DataFrame('brevet_doi.json')

In [ ]:
df_brevet_doi

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_brevet_doi, on='doi', how='left')

In [ ]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

In [ ]:
df_cache

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [ ]:
dict_doi_bso=[]

In [ ]:
for i,row in df_ipcc_ipbes_bso.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    dict_doi_bso.append(dict_row2)

In [ ]:
len(dict_doi_bso)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)

In [ ]:
with open(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes.jsonl', 'w') as f:
      f.write(json_lines)

elastic

In [ ]:
dd = pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes.jsonl', lines=True)

In [ ]:
dd2=dd.dropna(subset=['ipcc']).drop_duplicates(subset=['doi'])

In [ ]:
len(dd2.loc[(dd2['ipcc'].apply(lambda wg_chap: any('1' in x.get('wg', '') for x in wg_chap)))])

In [ ]:
current_dict = dd.to_dict(orient='records')

In [ ]:
list_bso_ipcc_ipbes=[]
for i in range(len(dd)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [ ]:
df_elastic_input=pd.DataFrame(current_dict)

In [ ]:
df_elastic_input.loc[:,'openalex_concepts']=df_elastic_input.loc[:,'openalex_concepts'].apply(type_score)

In [ ]:
del df_elastic_input['openalex_abstract_inverted_index']

In [ ]:
len(df_elastic_input)

In [ ]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

In [ ]:
#df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [ ]:
es_url_without_http = os.getenv("ES_URL").replace('https://','').replace('http://','')
es_host = f'https://{os.getenv("ES_LOGIN_TEDS_BACK")}:{parse.quote(os.getenv("ES_PASSWORD_TEDS_BACK"))}@{es_url_without_http}'
index_name='teds-bibliography'